In [1]:
import numpy as np
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
import pandas as pd
import torch
import random
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding,LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Preprocessing #

In [2]:
Sınıflandırıcı=pd.read_csv("Kat.csv") #verilerimi çektim

In [3]:
geçici=Sınıflandırıcı["category"]
geçici[60000]

'iade'

In [4]:
yorumlar=Sınıflandırıcı["text"] 
kategoriler=Sınıflandırıcı["category"]  #kategorilere one hot encoding işlemi yaptım
kategoriler= pd.Series(kategoriler)   
kategoriler=pd.get_dummies(kategoriler)
kategoriler=np.array(kategoriler)
yorumlar_tokenize=[] # tokenize edeceğiz sonra stopwordslerden ve köklerden kurtulacağız.
yorumlar=list(yorumlar)


In [5]:
kategoriler[3]

array([0, 0, 0, 0, 1, 0], dtype=uint8)

In [6]:
for cümle in yorumlar:
     yorumlar_tokenize.append(cümle.split())   #Tokenize yorumlarımı listeledim şimdi stopwordsları çıkaracağım.

In [7]:
dolgu_soz=[]

file = open("Stopwords.txt","r", encoding="utf-8")    #stopwords dosyamı aldım listeye akataracağm oradan da stopwordsleri atacağım"
icerik=file.read()
icerik=icerik.split('\n')


In [8]:
filtered_words = []                                #filtered words e aktarım yaptım
for word in yorumlar_tokenize:
    if word not in icerik:
        filtered_words.append(word)
yorumlar_tokenize=filtered_words


In [10]:
num_words = 10000                                                         #Kelimelerin bulunma sayılarına göre onları sayıya çevireceğiz                                                                          #Ondan sonra vektörleştireceğiz
tokenizer = Tokenizer(num_words=num_words)                                #sonra vektör haline getirilecek
tokenizer.fit_on_texts(yorumlar_tokenize)

In [ ]:
tokenizer.word_index

In [11]:
x = tokenizer.texts_to_sequences(yorumlar_tokenize)
                                     #burada kelimeleri sayıya çevirdik

In [12]:
num_tokens = [len(tokens) for tokens in x]
num_tokens = np.array(num_tokens)  #Burada ortalama sayıya baktım

Güzel ama bütün girdilerin aynı boyutta olması lazım

In [13]:
np.argmax(num_tokens)

86509

In [14]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens                        #Burada çoğunluğa yakın bir sayı belirledim ve fazla olanları atacak az olanlara 0 ekleyeceğim

125

In [15]:

from sklearn.model_selection import train_test_split    #test ve train setine böldük

x_train, x_test,y_train,y_test = train_test_split(x,kategoriler,test_size=0.20, random_state=0)
y_train=np.array(y_train)                      #Ağımıza vermek için np arrayine çevirdim.
y_test=np.array(y_test)

y_train

array([[0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0],
       ...,
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0]], dtype=uint8)

In [16]:
x_train_padded = pad_sequences(x_train, maxlen=max_tokens)   #padding işlemi yaptım
x_test_padded = pad_sequences(x_test, maxlen=max_tokens)

In [17]:
x_train_padded=np.array(x_train_padded)
x_test_padded=np.array(x_test_padded)

 # MODELİMİZİ KURALIM #

In [18]:
embedding_size = 50
model= Sequential()       

In [19]:
model.add(Embedding(input_dim=num_words,                     #NUM_WORDS İLE TOPLAM KELİME SAYISINI BELİRLEDİM 50 lik çıktı boyutu 
                     output_dim=embedding_size,             #verdim yani 50 boyutlu vektörler çıkacak input_length de belirlediğimiz
                        input_length=max_tokens,              #token sayısı
                    name='embedding_layer'))

Modelimde GRU kullanacağım GRU lar nlp modellerinde oldukça başarılılar
aktivasyon fonksiyonunu softmax olarak belirledim çünkü one hot vectorler için en iyisi bu
softmax bütün değerleri toplamı 1 olacak şekilde normalize ediyor

In [20]:

model.add(GRU(units=24))
model.add(Dense(6, activation='softmax'))                    

In [21]:
optimizer = Adam(lr=5*1e-3)                          #Optimizerımız Adam çünkü çok popüler :D learning ratemizide birkaç denemeden sonra girdim
model.compile(loss='categorical_crossentropy',     #Loss functionumuz categorical_crossentropy 
              optimizer=optimizer,
              metrics=['accuracy'])

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 125, 50)           500000    
_________________________________________________________________
gru (GRU)                    (None, 24)                5472      
_________________________________________________________________
dense (Dense)                (None, 6)                 150       
Total params: 505,622
Trainable params: 505,622
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.fit(x_train_padded, y_train, epochs=5, batch_size=256)    #256lık Batch size ile modelimizi eğittim acc değerimiz 0.8

Train on 72000 samples
Epoch 1/5
72000/72000 [==============================] - 47s 648us/sample - loss: 1.0184 - accuracy: 0.6271
Epoch 2/5
72000/72000 [==============================] - 45s 627us/sample - loss: 0.5781 - accuracy: 0.7811
Epoch 3/5
72000/72000 [==============================] - 43s 601us/sample - loss: 0.4637 - accuracy: 0.8240
Epoch 4/5
72000/72000 [==============================] - 45s 628us/sample - loss: 0.3761 - accuracy: 0.8611
Epoch 5/5
72000/72000 [==============================] - 47s 660us/sample - loss: 0.2956 - accuracy: 0.8940


In [24]:
result = model.evaluate(x_test_padded, y_test)      #test sonuçlarıda 0.78

18000/18000 [==============================] - 11s 592us/sample - loss: 0.6408 - accuracy: 0.7829


In [25]:
from keras import backend as K                         #Burası Kerasın hata vermesinden dolayı Stackoverflowdan bulduğum çözüm.
y_train = K.cast_to_floatx(y_train)
y_test=K.cast_to_floatx(y_test)

Using TensorFlow backend.


# Modeli ve veriyi modele uygun şekilde optimize etmek #

In [337]:
y_test_labels = np.argmax(y_test, axis =1)    

In [293]:
y_pred = model.predict(x=x_test_padded[:10])
y_pred = y_pred

In [339]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight('balanced' ,np.unique(y_test_labels) ,y_test_labels)  #buradan modelin dengesine baktım

In [340]:
class_weight  #vermemize gerek yok zaten gayet dengeli bir verisetimiz var

array([0.99833611, 0.99370653, 1.01936799, 0.98651759, 1.01798439,
       0.98522167])

In [30]:
import imblearn
from imblearn.under_sampling import InstanceHardnessThreshold

In [31]:

ncr = InstanceHardnessThreshold()
X_res, y_res = ncr.fit_sample(x_train_padded, y_train)


C:\Users\Onur\anaconda3\envs\tensorflow_env\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0 1 2 3 4 5] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


# Modelin Kullanımı #

In [26]:
idx = tokenizer.word_index
mapr = dict(zip(idx.values(), idx.keys()))
def tokens_to_string(tokens):
    words = [mapr[token] for token in tokens if token!=0]
    text = ' '.join(words)
    return text

In [27]:
tokens_to_string([2,3,45,6,345])    

'bir bu hizmetlerini kart kartın'

Şimdi Bir girdi verdiğimizde bize onun kategorisini söyleyen fonksiyonu yazalım

In [58]:
def string_to_text(string):   #Modele uygun hale getiren fonksiyonumuz hazır
    splitted=string.split()
    tokens = [str_To_Txt[token] for token in splitted]
    zero=0
    while(len(tokens)<max_tokens):tokens.append(zero)
    tokens=np.array(tokens)
    tokens=tokens.reshape(1,125)
    pred=model.predict(tokens)
    text3=one_hot_to_text(pred)
    
    return pred


In [29]:
def one_hot_to_text(ohv):   #Tahmini texte çeviriyor
    maks=np.argmax(ohv) #1 kredi kartı,iptal,hesap,musteri_hizmetleri,iade,kredi
    if(maks==4):
        d="kredi-kartı"
        return d
    if(maks==2):
        d="iptal"
        return d  
    if(maks==0):
        d="hesap"
        return d  
    if(maks==5):
        d="mus_hiz"
        return d
    if(maks==1):
        d="iade"
        return d  
    if(maks==3):
        d="kredi"
        return d

In [72]:
idx=tokenizer.word_index
str_To_Txt = dict(zip(idx.keys(), idx.values()))


In [31]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [32]:

widgets.Text(value='Bu Ufak ipynb uygulamasına hoşgeldin!', disabled=True)



Text(value='Bu Ufak ipynb uygulamasına hoşgeldin!', disabled=True)

In [129]:
a=widgets.Box(
       
    [   
        widgets.Button(
    description='Click me',
    disabled=False,
    button_style='info', 
    tooltip='Click me',
    icon='check' 
),
        
        widgets.Textarea(value="Sınıflandırmak İstediğin cümleyi gir")
        widgets.Label(value='Kategori:'),
        widgets.RadioButtons(
            options=[
                'Kredi-Kartı',
                'Kredi',
                'İptal',
                'hesap',
                'iade' 
            ],
            layout={'width': 'max-content'}
        )
    ]
)

SyntaxError: invalid syntax (<ipython-input-129-73a1e7733bf0>, line 13)

In [132]:
button = widgets.Button(description="Click Me!")
output = widgets.Output()
seçenek= widgets.RadioButtons(
            options=[
                'Kredi-Kartı',
                'Kredi',
                'İptal',
                'hesap',
                'iade'
            ]
                )
text_Box=widgets.Textarea(value="Sınıflandırmak İstediğin cümleyi gir")
display(button, output,seçenek,text_Box)

@output.capture()
def on_button_clicked(b):
    result=string_to_text(text_Box.value)
    print(result)
   

button.on_click(on_button_clicked)

Button(description='Click Me!', style=ButtonStyle())

Output()

RadioButtons(options=('Kredi-Kartı', 'Kredi', 'İptal', 'hesap', 'iade'), value='Kredi-Kartı')

Textarea(value='Sınıflandırmak İstediğin cümleyi gir')

'Sınıflandırmasgsdgdsgk İstediğin cümleyi gir'

In [40]:
onur=onur.split()
for a in onur:
    print(a)

hey
kredi
çekmek
istiyorum


In [75]:
model.predict(x_train_padded[44].reshape(1,125))

array([[1.2303501e-03, 4.5058745e-04, 7.4079544e-05, 9.9762923e-01,
        8.1433900e-05, 5.3437956e-04]], dtype=float32)

In [74]:
string_to_text( )

array([[0.8157809 , 0.04967908, 0.0018088 , 0.00242087, 0.0109795 ,
        0.11933084]], dtype=float32)